<a href="https://colab.research.google.com/github/ort-eila/csiseminar/blob/main/project/step1_preprocess_tif_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
csv_labels_file = "/content/gdrive/MyDrive/Seminar/project_candanace/patient_status.csv"
images_path = "/content/gdrive/MyDrive/Seminar/project_candanace/dcis_cohort/"
dataset_images_path = "/content/gdrive/MyDrive/Seminar/immune_310_project/"
root_dir = "/content/gdrive/MyDrive/Seminar/project/immune_310_project"

In [3]:
import os
print("csv_labels_file is {}".format(csv_labels_file))
print("images_path is {}".format(images_path))

csv_labels_file is /content/gdrive/MyDrive/Seminar/project_candanace/patient_status.csv
images_path is /content/gdrive/MyDrive/Seminar/project_candanace/dcis_cohort/


In [4]:
import pandas as pd

# Read the CSV file using pandas
df = pd.read_csv(csv_labels_file,index_col=False)

# Print the DataFrame (optional)
print(df.head())

   PointNumber         Status
0         2203  nonprogressor
1         2204  nonprogressor
2         2206  nonprogressor
3         2302  nonprogressor
4         2303  nonprogressor


In [5]:
df[df['PointNumber'] == 2203]['Status'].values[0]

'nonprogressor'

In [6]:
# collapse tif files to one file
def collapse_tif_files(path_to_files,point_number):
    collpase_file_path = ""
    return collpase_file_path

In [7]:
images_path

'/content/gdrive/MyDrive/Seminar/project_candanace/dcis_cohort/'

In [8]:
# organize the images based on labels
list_of_patients_points = os.listdir(images_path)
list_of_patients_points = [x for x in list_of_patients_points if ".DS" not in x and ".ipynb" not in x]
list_of_patients_points[0:5]

['Point4403', 'Point6103', 'Point4404', 'Point2301', 'Point2306']

In [9]:
# df[df['PointNumber'] == 4115]['Status'].values[0]

In [10]:
# dictionary of all the patients with image & label
point_dict = {}
for pp in list_of_patients_points:
    label = ""
    point = int(pp.split("_")[0].replace("Point", ""))
    # print("point is {}".format(point))
    # print(type(point))
    fld = os.path.join(images_path,pp)
    # Check if the specific PointNumber is available in the DataFrame
    is_available = point in df['PointNumber'].values
    if is_available is False:
        # print("point {} does not exist".format(point))
        continue
    else:
        label = df[df['PointNumber'] == point]['Status'].values[0]
    point_dict[str(point)] = {"fld":fld,"label":label}

In [11]:
len(point_dict) == 58

True

In [12]:
dataset_images_path

'/content/gdrive/MyDrive/Seminar/immune_310_project/'

In [13]:
# create folder for the images data npy in order
# dataset_images_path = os.path.join(root_dir,"Data/dataset_images")
print("dataset_images_path is {}".format(dataset_images_path))
os.makedirs(dataset_images_path,exist_ok=True)

dataset_images_path is /content/gdrive/MyDrive/Seminar/immune_310_project/


In [14]:
# images_channels_order

In [15]:
# get unique list of all the possible tif images
# get all the channels available for all patients and then run unique (set) the list
images_channels_order = []
remove_channels = ["P.tif","140empty.tif","Background.tif","Na.tif","Ca40.tif","C.tif","Ta.tif","Au.tif","ECADKRTCD45.tif","ECADKRTCD45GLUT1.tif","ECADKRTCD45GLUT1CD44.tif"]
for point, value in point_dict.items():
    # print("point is {} value is {}".format(point,value))
    raw_data_path = os.path.join(value['fld'],"TIFs")
    # print("raw_data_path is {}".format(raw_data_path))
    patient_tif_raw_images_list = os.listdir(raw_data_path) # list of all the channels of the patient
    # print("patient_tif_raw_images_list is {}".format(patient_tif_raw_images_list))
    # print("(len(original_tif_raw_images) before filter are {}".format(len(original_tif_raw_images)))
    patients_tif_channels = [item for item in patient_tif_raw_images_list if item not in remove_channels]
    # print("patient's_tif_channels are {}".format(patients_tif_channels))
    tif_raw_images = [x.split(".tif")[0] for x in patients_tif_channels]
    # print("after split tif_raw_images are {}".format(tif_raw_images))
    images_channels_order.extend(tif_raw_images)
# get unique values
print(len(images_channels_order))
images_channels_order = list(set(images_channels_order))
print(len(images_channels_order))


images_channels_order = [x+".tif" for x in images_channels_order if ".DS_Store" not in x]
# and "SMA" not in x and "PanKRT" not in x]
images_channels_order[0:4]

2378
41


['CD14.tif', 'HIF1a.tif', 'ECADKRT.tif', 'P63.tif']

In [16]:
import os
import numpy as np
from PIL import Image

def save_channels_as_npy(tif_path, tif_order, output_npy_path, width=1024, height=1024):
    channels = len(tif_order)
    channel_data = []

    # Loop through each channel and stack the data
    for i, tif_file in enumerate(tif_order):
        tif_file_path = os.path.join(tif_path, tif_file)
        if not os.path.isfile(tif_file_path):
            # Channel not found, fill with zeros
            channel_data.append(np.zeros((height, width), dtype=np.uint8))
        else:
            image_data = Image.open(tif_file_path)

            # Check if the image is already of the desired size (1024x1024)
            if image_data.size != (width, height):
                raise ValueError(f"Image {tif_file} is not of size {width}x{height}")

            # Convert the image to grayscale and stack it
            channel_data.append(np.array(image_data))

    # Stack the channel data into a single NumPy array
    channel_data = np.stack(channel_data, axis=0)

    # Save the channel data as an NPZ file
    np.savez(output_npy_path, channel_data=channel_data)


# New Section

In [17]:
# Iterate over key-value pairs in the dictionary
dataset_path=os.path.join(root_dir,"dataset","NPY_{}_channels".format(len(images_channels_order)),"labels")
# counter = 0 # for debug
# pil_image_stack=None
for point, value in point_dict.items():
    print(f"Point: {point}, Value: {value}")
    raw_data_path = os.path.join(value['fld'],"TIFs")
    print("raw_data_path is {}".format(raw_data_path))
    label_fld = os.path.join(dataset_path,value['label'])
    print("label_fld is {}".format(label_fld))
    os.makedirs(label_fld,exist_ok=True)
    output_path=os.path.join(label_fld,point+".npz")
    print("output_path is {}".format(output_path))
    save_channels_as_npy(raw_data_path, images_channels_order, output_path)


Point: 6103, Value: {'fld': '/content/gdrive/MyDrive/Seminar/project_candanace/dcis_cohort/Point6103', 'label': 'progressor'}
raw_data_path is /content/gdrive/MyDrive/Seminar/project_candanace/dcis_cohort/Point6103/TIFs
label_fld is /content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels/progressor
output_path is /content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels/progressor/6103.npz
Point: 2306, Value: {'fld': '/content/gdrive/MyDrive/Seminar/project_candanace/dcis_cohort/Point2306', 'label': 'progressor'}
raw_data_path is /content/gdrive/MyDrive/Seminar/project_candanace/dcis_cohort/Point2306/TIFs
label_fld is /content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels/progressor
output_path is /content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels/progressor/2306.npz
Point: 2309, Value: {'fld': '/content/gdrive/MyDrive/Seminar/project_candanace/dci